# Ансамбли. Бэггинг и случайный лес

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
SEED = 314159
TRAIN_TEST_SPLIT = 0.80

data_path = "D:\data\machine_learning"

Деревья решений имеют так называемый низкий предвзятость/сдвиг (bias) и высокую дисперсию (variance). В результате модели на их основе подвержены переобучению, но в целом достаточно точны. Это свойство деревьев решений активно эксплуатируется в ансамблях (где используются и другие модели, но чаще всего именно они)
Рассмотрим разные способы ансамблирования.

# Bagging
Бэггинг (bootstrap aggregating) — это ансамблевый метод, который включает в себя независимое обучение нескольких моделей на случайных подмножествах данных и агрегирование их прогнозов посредством голосования или усреднения.

С помощью бэггинга работают и случайные леса. Идея в том, чтобы последовательно получить выборку с возвращением n раз и обучить n базовых моделей $b_i(x) = b(x, X^i)$.
Предсказание результрующей модели будут выглядеть как $a(x) = \frac{1}{k}(b_1(x) + \dots + b_k(x)).$

Этапы построения ансамля:
1) Создается бутстрапированная выборка
2) На ней учится дерево решений
3) Для финального предсказания используется среднее всех деревьев

Вопрос: будет ли бэггинг подвержен оверфиттингу, как одно решающее дерево?


In [ ]:
df = pd.read_csv(data_path+'/'+"winequality-red.csv")
df_major = df[df["quality"].isin([5,6])]
print("Length of filtered data is", len(df_major))
X = df_major.drop('quality', axis=1)
y = df_major['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=SEED)

In [ ]:
model = DecisionTreeClassifier(random_state=SEED)
model.fit(X_train, y_train)
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

In [ ]:
print(y_train.value_counts())

Фактически мы можем вычислить, что вероятность того, что наблюдение будет исключено из нашего набора данных с бутсрапированием, равна $(1 - \frac{1}{n})^{n}$.
По определению $e^{-1} = \displaystyle \lim_{n\to\infty}(1-\frac{1}{n})^n$ и так как $e^{-1} ~ 0.36$, то мы выкидываем примерно треть всех данных в каждом дереве. То есть получается, что каждое дерево, построенное на такой выборке, будет достаточно сильно отличаться от остальных. Будет ли этого достаточно для получения нескоррелированных моделей?

In [ ]:
from sklearn.utils import resample
X_train_bs, y_train_bs = # resamle data once or twice to check that final sample differs
# your code

## Bagging
Бэггинг — это процесс выращивания дерева, при котором каждый узел дерева просматривает каждый признак в нашей бутстрапированной выборке, чтобы найти наилучшее разделение данных в этом конкретном узле. Это повторяется для всех деревьев.

In [ ]:
from sklearn.ensemble import BaggingClassifier
model = # create decision tree
# Create a bagging classifier with the decision tree pipeline
bagging_classifier = BaggingClassifier(estimator=model, n_estimators=50, random_state=42)

# Train the bagging classifier on the training data
bagging_classifier.fit(X_train, y_train)
print("Train score is: ", bagging_classifier.score(X_train, y_train))
print("Test score is: ", bagging_classifier.score(X_test, y_test))

В бэггинге делается предположение, что базовые модели некоррелированы, и за счёт этого получается очень сильное уменьшение дисперсии у ансамбля относительно базовых моделей. Однако в реальной жизни добиться этого сложно: ведь базовые алгоритмы учили одну и ту же зависимость на пересекающихся выборках.
На практике оказывается, что строгое выполнение этого предположения не обязательно. Достаточно, чтобы модели были в некоторой степени не похожи друг на друга. На этом облегчении строится развитие идеи бэггинга для решающих деревьев — случайный лес.

Главное отличие случайного леса от bagging-а в том, что в случайном лесе рассматривается только $m=\sqrt(p)$ признаков. Благодаря этому мы сможем создавать множество *некоррелированных* деревьев, которые помогут нам уловить большую часть изменчивости, а также взаимодействия между несколькими переменными.

In [ ]:
# fit and get scores for random forest

Для того, чтобы лучше понять, как устроен случайный лес, сделаем свою (упрощенную) версию этой модели.

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
class RandomForest(BaseEstimator, ClassifierMixin):
    def __init__(self, n_estimators: int = 10, random_state: int = 0, max_features="sqrt",**kwargs):
        super().__init__()
        self.max_features = max_features
        # init forest
        self._estimators = []


    def fit(self, X, y):
        if self.max_features == "sqrt":
            # set max features as sqrt of number of features
        elif self.max_features == "log":
            # set max features as log2 of number of features
        else:
            self.max_features = None

        for i in range(self.n_estimators):
            # fit estimators

    def predict(self, X):
        predicts = []
        # get predict from every estimator
        predicts = np.array(predicts).T
        predicts = np.array([np.argmax(np.bincount(predicts[i])) for i in range(len(X))])
        return predicts


In [ ]:
# fit your random forest

In [ ]:
print("Train score is: ", model.score(X_train, y_train))
print("Test score is: ", model.score(X_test, y_test))

# OOB test
Еще одна замечательная особенность бутсреппинга заключается в том, что мы бесплатно получаем так называемую out-of-bag оценку ошибок. Выборка OOB (out of bag) — это ≈1/3 наблюдений, которые не были выбраны для построения конкретного дерева. После того, как мы построили наше дерево с помощью n наблюдений, мы можем проверить каждый оставшийся $x_i$ и в итоге вычислить среднюю ошибку прогнозирования на основе этого набора.

Более того, мы можем вычислить оценку OOB для каждого дерева и взять среднее значение всех этих оценок, чтобы получить оценку точности работы нашего случайного леса. По сути, это перекрестная проверка с исключением (leave-one-out). Это даст нам оценку точности нашей модели без необходимости формального тестирования ее на новых данных.

В sklearn oob score можно использовать вместо проверки на тестовом множестве

In [ ]:
# fit standart rf
print("Train oob score is: ", model.oob_score_)

# Bias-variance
Напомню, что ошибка модели (на которую мы можем повлиять) состоит из смещения и разброса. До этого мы предполагали (и могли показать теоретически), что случайный лес позволяет уменьшить разброс/дисперсию.
Вспомним, как мы раньше оценивали bias-variance составляющие. Проверим, действительно ли случайный лес ведет себя так, как мы предполагали.

In [ ]:
from sklearn.metrics import zero_one_loss
from mlxtend.evaluate import bias_variance_decomp
# get decombosed error (from introduction notebook)

print('Average expected loss: %.3f' % avg_expected_loss)
print('Average bias: %.3f' % avg_bias)
print('Average variance: %.3f' % avg_var)
print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred))



И для целого леса

In [ ]:
# get decombosed error for random forest (from introduction notebook)


## Число деревьев
Увеличение числа элементарных моделей в ансамбле не меняет смещения и уменьшает дисперсию. Вопрос. Почему бы не использовать всегда огромное число деревьев?
Мы можем даже посмотреть, как меняются оценки ошибок при добавлении новых деревьев. Заодно сравним разные способы ограничения количества признаков.

In [ ]:
from sklearn.model_selection import cross_val_score
from collections import OrderedDict

def get_score_plots(X_train, y_train, oob=True):
    # NOTE: Setting the `warm_start` construction parameter to `True` disables
    # support for parallelized ensembles but is necessary for tracking the OOB
    # error trajectory during training.
    params = dict(warm_start=True, oob_score=oob,random_state=SEED)
    ensemble_clfs = [
        (
            "RandomForestClassifier, max_features='sqrt'",
            RandomForestClassifier(max_features="sqrt", **params),
        ),
        (
            "RandomForestClassifier, max_features='log2'",
            RandomForestClassifier(max_features="log2", **params),
        ),
        (
            "RandomForestClassifier, max_features=None",
            RandomForestClassifier(max_features=None,**params),
        ),
    ]

    error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

    # Range of `n_estimators` values to explore.
    min_estimators = 20
    max_estimators = 150

    for label, clf in ensemble_clfs:
        for i in range(min_estimators, max_estimators + 1, 5):
            clf.set_params(n_estimators=i)
            error = # fit classifier and compute error rate (1 - error)
            error_rate[label].append((i, error))
    # Generate the "OOB error rate" vs. "n_estimators" plot.
    for label, clf_err in error_rate.items():
        xs, ys = zip(*clf_err)
        plt.plot(xs, ys, label=label)

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("n_estimators")
    y_label_pre = "OOB" if oob else "Test"
    plt.ylabel(y_label_pre +" error rate")
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
get_score_plots(X_train, y_train)

In [ ]:
get_score_plots(X_train, y_train, oob=False)

А что с нашим случайным лесом?

In [ ]:
def get_our_plots(X_train, y_train, oob=True):
    clf = RandomForest(random_state=SEED)
    error_rate = []
    # Range of `n_estimators` values to explore.
    min_estimators = 20
    max_estimators = 150

    # your code. get values and generate the "error rate" vs. "n_estimators" plot.

    xs, ys = zip(*error_rate)
    plt.plot(xs, ys, label="Our random forest")

    plt.xlim(min_estimators, max_estimators)
    plt.xlabel("n_estimators")
    y_label_pre = "OOB" if oob else "Test"
    plt.ylabel(y_label_pre +" error rate")
    plt.legend(loc="upper right")
    plt.show()

In [ ]:
get_our_plots(X_train, y_train)

## Какая должна быть глубина деревьев в случайном лесу?
Разброс уменьшается с помощью бэггинга. На смещение бэггинг не влияет, а хочется, чтобы у леса оно было небольшим. Поэтому смещение должно быть небольшим у самих деревьев, из которых строится ансамбль. У неглубоких деревьев малое число параметров, поэтому они могут запомнить самые простые статистики. К каким значениям смещения и дисперсии это приводит?
И какой глубины все же выбирать деревья?

In [ ]:
def get_error_dec(clf_dt, depth: int = 2):
    # your code. decompose error
    print("Depth = ", depth)
    print('Average expected loss: %.3f' % avg_expected_loss)
    print('Average bias: %.3f' % avg_bias)
    print('Average variance: %.3f' % avg_var)
    print('Sklearn 0-1 loss: %.3f' % zero_one_loss(y_test,y_pred))
get_error_dec(depth=2)

In [ ]:
# prepare and fit decision tree
get_error_dec(depth=8)

In [ ]:
# prepare and fit random forest. get error decomposition


In [ ]:
# if we have time, prepare plotsof bias-variance for different depths and numbers of trees

## Границы разбиения
Рассмотрим также и границы принятия решений. То, как они будут выглядеть, становится довольно тривиальным вопросом, однако все равно построим их, чтобы полностью прочуствовать то, как работает RandomForest.

In [ ]:
# copy the plotting code from previous practice

In [ ]:
# copy the plotting code from previous practice

In [ ]:
# copy the plotting code from previous practice
plot_boundary(classifier, data=X_train_pca, features=features, y=y_train)

In [ ]:
features = ["x1", "x2"]
classifier = # random forest
plot_boundary(classifier, data=X_train_pca, features=features, y=y_train)

In [ ]:
# you can plot decision boundary for bagging classifier too.

## Bagging для линейных моделей.
Если останется время, давайте рассмотрим и другое семейство моделей в качестве базовых. Вопрос. Как будет выглядеть предсказание для такого случая?

In [ ]:
# what is the decision boundary of linear models ensemble?